<a href="https://colab.research.google.com/github/AlbaCastillo/Movie-Recommendations-Model/blob/main/Movie_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movies Recommendations


### ETL

In [1]:
import pandas as pd
import numpy as np

Carga de datos

In [2]:
# Se extrae directamente desde un archivo en la carpeta de drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Se cargan los datos a un dataframe
movies = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/movies_dataset.csv")

<ipython-input-3-ae0dddb60e7b>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/movies_dataset.csv")


In [4]:
# movies = pd.read_csv("/movies_dataset.csv")

In [5]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [6]:
movies_final = movies

In [7]:
movies_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Valores nulos de los campos revenue, budget rellenados por el número 0

In [8]:
movies_final['revenue'].fillna(0, inplace=True)

In [9]:
movies_final['budget'].fillna(0, inplace=True)

 Valores nulos del campo release_date  deben eliminarse

In [10]:
movies_final.dropna(subset=['release_date'], inplace=True)

Columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget

In [11]:
# movies_final['return'] = movies_final['revenue'].astype(float) / movies_final['budget'].astype(float)
# ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [12]:
# Mascara del valor erroneo
# mask = movies_final[(movies_final['revenue'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg') | (movies_final['budget'] == '/ff9qCepilowshEtG2GYWwzt2bs4.jpg')]

In [13]:
# Borrar fila
# movies_final = movies_final.drop(mask.index)

Se observa un error en los datos de la columna 'budget' ya que deberia ser el presupuesto de la película, en dólares. Y hay datos tipo '/ ... .jpg'

Son varias columnas entonces:

In [14]:
# Convertir valores numéricos a float y mantener valores de texto como string
movies_final['budget'] = pd.to_numeric(movies_final['budget'], errors='coerce').fillna(movies_final['budget'])
#El parámetro "errors" se establece en "coerce" para que cualquier valor que no se pueda convertir en un número se establezca en "NaN"
# Luego se rellena con el valor original fillna(movies_final['budget']), para luego ser borrados

In [15]:
# Errores
error = movies_final.loc[movies_final['budget'].apply(type) == str, 'budget']
error

19730    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
29503    /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
35587    /zaSf5OG7V8X8gqFvly88zDdRm46.jpg
Name: budget, dtype: object

In [16]:
# Borrar filas
movies_final = movies_final.drop(error.index)

In [17]:
movies_final['budget'] == 0

0        False
1        False
2         True
3        False
4         True
         ...  
45460     True
45462     True
45463     True
45464     True
45465     True
Name: budget, Length: 45376, dtype: bool

In [18]:
(movies_final['budget'] < 0).any()

False

In [19]:
# Calcular la columna "return" y asignar 0 cuando "budget" sea 0 ya que no se puede dividir entre 0
movies_final['return'] = np.where(movies_final['budget'] == 0, 0,
                                  movies_final['revenue'].astype(float) / movies_final['budget'].astype(float))

In [20]:
movies_final['return']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45376, dtype: float64

Columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage

In [21]:
Cols = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_final = movies_final.drop(Cols, axis=1)

In [22]:
movies_final.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801


Fechas, con el formato AAAA-mm-dd

In [23]:
movies_final['release_date'] = pd.to_datetime(movies_final['release_date'])
movies_final['release_date'] = movies_final['release_date'].dt.strftime('%Y-%m-%d')


In [24]:
movies_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  object 
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

 Crear la columna release_year donde se extraera el año de la fecha de estreno

In [25]:
movies_final['release_date'] = pd.to_datetime(movies_final['release_date'])
movies_final['release_year'] = movies_final['release_date'].dt.year

Desanidar Datos

belongs_to_collection


In [26]:
# Se extrae name de belongs_to_collection
# movies_final[['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path']] = movies_final['belongs_to_collection'].str.extract(r"{'id': (\d+), 'name': '(.+?)', 'poster_path': '(.+?)', 'backdrop_path': '(.+?)'}")
movies_final['collection_name'] = movies_final['belongs_to_collection'].str.extract("'name': '([^']*)'")

In [27]:
movies_final.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,collection_name
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,Toy Story Collection


In [28]:
# Eliminamos belongs_to_collection
movies_final = movies_final.drop('belongs_to_collection', axis=1)

genres

In [29]:
movies_final.loc[1, 'genres']

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [30]:
import ast
# Convertimos la cadena en una lista de diccionarios
movies_final['genres'] = movies_final['genres'].apply(ast.literal_eval)

In [31]:
# Función para extraer los nombres de los géneros
def extract_names(genres_list):
    genres_names = [genre['name'] for genre in genres_list]
    genres_names = ', '.join(genres_names)
    return genres_names

# Aplicamos la función a cada elemento de la columna 'genres'
movies_final['genres_name'] = movies_final['genres'].apply(extract_names)

In [32]:
movies_final.head(1)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,collection_name,genres_name
0,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,Toy Story Collection,"Animation, Comedy, Family"


In [33]:
# Eliminamos genres
movies_final = movies_final.drop('genres', axis=1)

production_companies

In [34]:
movies_final.loc[3, 'production_companies']

"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]"

In [35]:
# Convertimos la cadena en una lista de diccionarios
movies_final['production_companies'] = movies_final['production_companies'].apply(ast.literal_eval)

In [36]:
# Aplicamos la función extract_names a cada elemento de la columna 'production_companies'
movies_final['production_companies_names'] = movies_final['production_companies'].apply(extract_names)

In [37]:
movies_final.loc[3, 'production_companies_names']

'Twentieth Century Fox Film Corporation'

In [38]:
# Eliminamos production_companies
movies_final = movies_final.drop('production_companies', axis=1)

production_countries

In [39]:
movies_final.loc[4, 'production_countries']

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [40]:
# Convertimos la cadena en una lista de diccionarios
movies_final['production_countries'] = movies_final['production_countries'].apply(ast.literal_eval)

In [41]:
# Aplicamos la función extract_names a cada elemento de la columna 'production_countries'
movies_final['production_country'] = movies_final['production_countries'].apply(extract_names)

In [42]:
# Eliminamos 'production_countries'
movies_final = movies_final.drop('production_countries', axis=1)

spoken_languages

In [43]:
movies_final.loc[3, 'spoken_languages']

"[{'iso_639_1': 'en', 'name': 'English'}]"

In [44]:
# Convertimos la cadena en una lista de diccionarios
movies_final['spoken_languages'] = movies_final['spoken_languages'].apply(ast.literal_eval)

In [45]:
# Aplicamos la función extract_names a cada elemento de la columna 'spoken_languages'
movies_final['spoken_languages_names'] = movies_final['spoken_languages'].apply(extract_names)

In [46]:
movies_final.loc[3, 'spoken_languages_names']

'English'

In [47]:
# Eliminamos spoken_languages
movies_final = movies_final.drop('spoken_languages', axis=1)

In [48]:
movies_final.head(1)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,return,release_year,collection_name,genres_name,production_companies_names,production_country,spoken_languages_names
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,Toy Story Collection,"Animation, Comedy, Family",Pixar Animation Studios,United States of America,English


In [49]:
movies_final.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'return', 'release_year',
       'collection_name', 'genres_name', 'production_companies_names',
       'production_country', 'spoken_languages_names'],
      dtype='object')

In [50]:
movies_final = movies_final.drop_duplicates()

In [51]:
movies_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45363 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   budget                      45363 non-null  object        
 1   id                          45363 non-null  object        
 2   original_language           45352 non-null  object        
 3   overview                    44422 non-null  object        
 4   popularity                  45363 non-null  object        
 5   release_date                45363 non-null  datetime64[ns]
 6   revenue                     45363 non-null  float64       
 7   runtime                     45117 non-null  float64       
 8   status                      45283 non-null  object        
 9   tagline                     20394 non-null  object        
 10  title                       45363 non-null  object        
 11  vote_average                45363 non-null  float64   

In [86]:
movies_final.columns


Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'return', 'release_year',
       'collection_name', 'genres_name', 'production_companies_names',
       'production_country', 'spoken_languages_names'],
      dtype='object')

In [89]:
# Dejamos solos los campos que se van a usar para la API
movies_final_op = movies_final[['budget', 'id', 'popularity', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'return', 'release_year']]

Credits

In [52]:
credits = pd.read_csv("/content/drive/MyDrive/Modelo de Recomendaciones (Movies)/credits.csv")

In [53]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [54]:
credits.shape

(45476, 3)

In [55]:
credits_final = credits

In [56]:
credits_final.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [57]:
credits_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [58]:
credits_final.duplicated().any()

True

In [59]:
credits_final.drop_duplicates().reset_index(drop=True)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45434,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45435,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45436,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45437,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [60]:
# Convertimos la cadena en una lista de diccionarios
credits_final['cast'] = credits_final['cast'].apply(ast.literal_eval)

In [61]:
# Aplicamos la función extract_names a cada elemento de la columna 'cast_name'
credits_final['cast_name'] = credits_final['cast'].apply(extract_names)

In [62]:
# Convertimos la cadena en una lista de diccionarios
credits_final['crew'] = credits_final['crew'].apply(ast.literal_eval)

In [63]:
# 'job': 'Director', 'name': 'John Lasseter'

# Función para extraer los nombres cuando job = Director
def extract_director_name(crew_list):
    director_names = []
    for crew in crew_list:
        if crew['job'] == 'Director':
            director_names.append(crew['name'])
            break
    director_name = ', '.join(director_names)
    return director_name

In [64]:
credits_final['director'] = credits_final['crew'].apply(extract_director_name)

In [65]:
credits_final.head(1)

,cast,crew,id,cast_name,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter


In [66]:
# Eliminamos cast y crew
credits_final = credits_final.drop(['cast', 'crew'], axis=1)

In [67]:
credits_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         45476 non-null  int64 
 1   cast_name  45476 non-null  object
 2   director   45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


## Consultas

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [68]:
def cantidad_filmaciones_mes( Mes ):
  mes = Mes.lower()
  meses = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
  mes_nro = meses.get(mes)
  estrenos = movies_final[movies_final['release_date'].dt.month == mes_nro]
  cantidad = len(estrenos['id'].unique())
  return {'mes':mes.capitalize(), 'cantidad':cantidad}


In [69]:
cantidad_filmaciones_mes('JULIO')

{'mes': 'Julio', 'cantidad': 2638}

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [70]:
def cantidad_filmaciones_dia( Dia ):
  dia = Dia.lower()
  dias = {'lunes': 'Monday', 'martes': 'Tuesday', 'miercoles': 'Wednesday', 'jueves': 'Thursday', 'viernes': 'Friday', 'sabado': 'Saturday', 'domingo': 'Sunday'}
  dia_en = dias.get(dia)
  estrenos = movies_final[movies_final['release_date'].dt.day_name() == dia_en]
  cantidad = len(estrenos['id'].unique())
  return {'dia':dia.capitalize(), 'cantidad':cantidad}

In [71]:
cantidad_filmaciones_dia('VIERNES')

{'dia': 'Viernes', 'cantidad': 13902}

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

In [72]:
def score_titulo( titulo_de_la_filmación ):
  title = titulo_de_la_filmación.replace(" ", "").lower()
  movie_c = movies_final[movies_final['title'].str.replace(" ", "").str.lower().str.contains(title)].drop_duplicates(subset='id')
  if movie_c.empty:
        return f'No se encontró la película {titulo_de_la_filmación} en la base de datos.'
  movie = movie_c['title'].iloc[0]
  year = movie_c['release_year'].iloc[0]
  popularity = movie_c['popularity'].iloc[0]
  return {'titulo':movie, 'anio':year, 'popularidad':popularity}

In [73]:
score_titulo('toystory')

{'titulo': 'Toy Story', 'anio': 1995, 'popularidad': 21.946943}

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [74]:
def votos_titulo( titulo_de_la_filmación ):
  title = titulo_de_la_filmación.replace(" ", "").lower()
  movie_c = movies_final[movies_final['title'].str.replace(" ", "").str.lower().str.contains(title)].drop_duplicates(subset='id')
  if movie_c.empty:
        return f'No se encontró la película {titulo_de_la_filmación} en la base de datos.'
  if (movie_c['vote_count'].iloc[0]<2000):
    return f'La película {titulo_de_la_filmación} debe tener al menos 2000 valoraciones para ser medida'
  #vote_average	vote_count
  movie = movie_c['title'].iloc[0]
  year = movie_c['release_year'].iloc[0]
  vote_count = movie_c['vote_count'].iloc[0]
  vote_average = movie_c['vote_average'].iloc[0]
  return  {'titulo':movie, 'anio':year, 'voto_total':vote_count, 'voto_promedio':vote_average}

In [75]:
votos_titulo(' toy  stOry')

{'titulo': 'Toy Story',
 'anio': 1995,
 'voto_total': 5415.0,
 'voto_promedio': 7.7}

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [76]:
credits_final.columns

Index(['id', 'cast_name', 'director'], dtype='object')

In [77]:
def get_actor( nombre_actor ):
  credits_final.dropna(subset=['cast_name'], inplace=True)
  actor = nombre_actor.replace(" ", "").lower()
  mask = credits_final['cast_name'].str.replace(" ", "").str.lower().str.contains(actor)
  # Filtrar el DataFrame para mostrar solo las filas donde se encontró el nombre
  filtered = credits_final[mask]
  filtered_movies = movies_final[(movies_final['id'].astype(int)).isin(filtered['id'].astype(int))].drop_duplicates(subset='id')
  cantidad = len(filtered)
  return_movies = round(filtered_movies['return'].sum(), 3)
  if cantidad > 0:
      average = round(return_movies / cantidad, 3)
  else:
      average = 0
  return {'actor':nombre_actor.title(), 'cantidad_filmaciones':cantidad, 'retorno_total':return_movies, 'retorno_promedio':average}


In [78]:
get_actor('tom hanks')

{'actor': 'Tom Hanks',
 'cantidad_filmaciones': 71,
 'retorno_total': 178.849,
 'retorno_promedio': 2.519}

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma

In [79]:
movies_final.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'return', 'release_year',
       'collection_name', 'genres_name', 'production_companies_names',
       'production_country', 'spoken_languages_names'],
      dtype='object')

In [80]:
def get_director( nombre_director ):
  credits_final.dropna(subset=['director'], inplace=True)
  director = nombre_director.replace(" ", "").lower()
  mask = credits_final['director'].str.replace(" ", "").str.lower().str.contains(director)
  # Filtrar el DataFrame para mostrar solo las filas donde se encontró el nombre
  filtered = credits_final[mask]
  filtered_movies = movies_final[(movies_final['id'].astype(int)).isin(filtered['id'].astype(int))].drop_duplicates(subset='id')
  return_movies = round(filtered_movies['return'].sum(), 3)
  movies = ', '.join(filtered_movies['title'].astype(str))
  year = ', '.join(filtered_movies['release_year'].astype(str))
  return_movie = ', '.join(filtered_movies['return'].astype(str).apply(lambda x: str(round(float(x), 3))))
  budget = ', '.join(filtered_movies['budget'].astype(str).apply(lambda x: str(round(float(x), 3))))
  revenue = ', '.join(filtered_movies['revenue'].astype(str).apply(lambda x: str(round(float(x), 3))))

  return  {'director':nombre_director.title(), 'retorno_total_director':return_movies,
    'peliculas':movies, 'anio':year, 'retorno_pelicula':return_movie,
    'budget_pelicula':budget, 'revenue_pelicula':revenue}

In [81]:
get_director('John Lasseter')

{'director': 'John Lasseter',
 'retorno_total_director': 27.654,
 'peliculas': "Toy Story, A Bug's Life, Toy Story 2, Luxo Jr., Cars, Cars 2, Tin Toy, Red's Dream, Knick Knack, Mater and the Ghostlight",
 'anio': '1995, 1998, 1999, 1986, 2006, 2011, 1988, 1987, 1989, 2006',
 'retorno_pelicula': '12.452, 3.027, 5.526, 0.0, 3.85, 2.799, 0.0, 0.0, 0.0, 0.0',
 'budget_pelicula': '30000000.0, 120000000.0, 90000000.0, 0.0, 120000000.0, 200000000.0, 0.0, 0.0, 0.0, 0.0',
 'revenue_pelicula': '373554033.0, 363258859.0, 497366869.0, 0.0, 461983149.0, 559852396.0, 0.0, 0.0, 0.0, 0.0'}

In [82]:
movies_final.to_csv('movies_final.csv', index=False)

In [83]:
from google.colab import files
# files.download('movies_final.csv')

In [84]:
credits_final.to_csv('credits_final.csv', index=False)

In [85]:
# files.download('credits_final.csv')

In [90]:
movies_final_op.to_csv('movies_final_op.csv', index=False)

In [91]:
#files.download('movies_final_op.csv',)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>